<div style="text-align: center;">
<h1>The University of North Carolina at Chapel Hill</h1>
<h1>Comp 555 BioAlgorithms - Spring 2018</h1>
<h1 style="font-size: 250%;">Problem Set #3 </h1>
<h1>Issued Thursday, 3/1/2018; Due Thursday, 3/22/2018</h1>
</div>

**Homework Information:** Some of the problems are probably too long to be started the night before the due date, so plan accordingly. Late problem sets will be penalized by a factor of	70.71% for each class meeting after the due date. Feel free to get help from others, but the work you submit in should be your own.

In [1]:
# Replace the following string values with the requested information
class Student:
    first = ""
    last = ""
    onyen = ""
    pid = ""

---
**Problem #1:** A file of *15-mers* simulating short reads from a genome can be downloaded [here](http://csbio.unc.edu/mcmillan/Comp555S18/data/kmers.txt). How many distinct nodes appear in the De Bruijn graph that represents these *15-mers* as edges? How many nodes are semi-balenced? How many nodes are balanced? How many are balanced with both in-degrees and out-degrees equal to 1?  

In [15]:
import itertools

class Graph:
    def __init__(self, vlist=[]):
        """ Initialize a Graph with an optional vertex list """
        self.index = {v:i for i,v in enumerate(vlist)}
        self.vertex = {i:v for i,v in enumerate(vlist)}
        self.edge = []
        self.edgelabel = []
    def addVertex(self, label):
        """ Add a labeled vertex to the graph """
        index = len(self.index)
        self.index[label] = index
        self.vertex[index] = label
    def addEdge(self, vsrc, vdst, label='', repeats=True):
        """ Add a directed edge to the graph, with an optional label. 
        Repeated edges are distinct, unless repeats is set to False. """
        e = (self.index[vsrc], self.index[vdst])
        if (repeats) or (e not in self.edge):
            self.edge.append(e)
            self.edgelabel.append(label)
    def degrees(self):
        """ Returns two dictionaries with the inDegree and outDegree
        of each node from the graph. """
        inDegree = {}
        outDegree = {}
        for src, dst in self.edge:
            outDegree[src] = outDegree.get(src, 0) + 1
            inDegree[dst] = inDegree.get(dst, 0) + 1
        return inDegree, outDegree
    def verifyAndGetStart(self):
        inDegree, outDegree = self.degrees()
        start = 0
        end = 0
        for vert in self.vertex.iterkeys():
            ins = inDegree.get(vert,0)
            outs = outDegree.get(vert,0)
            if (ins == outs):
                continue
            elif (ins - outs == 1):
                end = vert
            elif (outs - ins == 1):
                start = vert
            else:
                start, end = -1, -1
                break
        if (start >= 0) and (end >= 0):
            return start
        else:
            return -1
    def eulerianPath(self):
        graph = [(src,dst) for src,dst in self.edge]
        currentVertex = self.verifyAndGetStart()
        path = [currentVertex]
        # "next" is where vertices get inserted into our tour
        # it starts at the end (i.e. it is the same as appending),
        # but later "side-trips" will insert in the middle
        next = 1
        while len(graph) > 0:
            for edge in graph:
                if (edge[0] == currentVertex):
                    currentVertex = edge[1]
                    graph.remove(edge)
                    path.insert(next, currentVertex)
                    next += 1
                    break
            else:
                for edge in graph:
                    try:
                        next = path.index(edge[0]) + 1
                        currentVertex = edge[0]
                        break
                    except ValueError:
                        continue
                else:
                    print "There is no path!"
                    return False
        return path
    def eulerEdges(self, path):
        edgeId = {}
        for i in xrange(len(self.edge)):
            edgeId[self.edge[i]] = edgeId.get(self.edge[i], []) + [i]
        edgeList = []
        for i in xrange(len(path)-1):
            edgeList.append(self.edgelabel[edgeId[path[i],path[i+1]].pop()])            
        return edgeList

with open("kmers.txt") as f:
    content = f.readlines()
content = [x.strip() for x in content] 
print len(content)
verts = []

for v in content:
    if v[0:len(v)-1] not in verts:
        verts.append(v[0:len(v)-1])
    if v[1:len(v)] not in verts:
        verts.append(v[1:len(v)])
print len(verts)
print verts[0],verts[1]

93925
93674
AAAAAAAAGCCCGG AAAAAAAGCCCGGG


In [19]:
g = Graph(verts)
for v in content:
    g.addEdge(v[0:len(v)-1],v[1:len(v)],v)
d = g.degrees()
inn = d[0]
out = d[1]
semi = 0
bal = 0
eq = 0 
for key in out:
    value = out[key]
    if abs(value - inn.get(key,0)) ==1:
        semi +=1
for key in inn:
    value = inn[key]
    if abs(value - out.get(key,0)) ==1:
        semi +=1
    if abs(value - out.get(key,0)) ==0:
        bal +=1 
    if abs(value - out.get(key,0)) ==0 and value == 1 and out.get(key,0) == 1:
        eq +=1 
print semi,bal,eq

2 93672 93522


---
**Problem #2:** What is the length of the Eulerian path that can be constructed in the De Bruijn graph described in Problem #1? How does the resulting constructed sequence compare to the plasmid sequence of [*Salmonella Typhimurium*](http://csbio.unc.edu/mcmillan/Comp555S18/data/SalmonellaTyphimurium.fa). (Hint: one method of comparison is consider how many k-mers the two sequences share. Of course you can compare where k=15, but consider what is the smallest value of k for which the two sequences differ in their set of k-mers?)

In [5]:
eulerpath = g.eulerianPath()
print len(eulerpath)

93926


In [39]:
def kmerFreq(k, sequence):
    kmerCount = {}
    for i in xrange(len(sequence)-k+1):
        kmer = sequence[i:i+k]
        kmerCount[kmer] = kmerCount.get(kmer,0)+1
    return kmerCount
def loadFasta(filename):
    """ Parses a classically formatted and possibly 
        compressed FASTA file into a list of headers 
        and fragment sequences for each sequence contained"""
    if (filename.endswith(".gz")):
        fp = gzip.open(filename, 'rb')
    else:
        fp = open(filename, 'rb')
    # split at headers
    data = fp.read().split('>')
    fp.close()
    # ignore whatever appears before the 1st header
    data.pop(0)     
    headers = []
    sequences = []
    for sequence in data:
        lines = sequence.split('\n')
        headers.append(lines.pop(0))
        # add an extra "+" to make string "1-referenced"
        sequences.append('+' + ''.join(lines))
    return (headers, sequences)

header, seq = loadFasta("SalmonellaTyphimurium.fa")
seq = seq[1]
seq = seq[1:len(seq)]

euleredges = g.eulerEdges(eulerpath)
path = euleredges[0]
for i in range(1,len(euleredges)):
    path = path + euleredges[i][len(euleredges[i])-1:len(euleredges[i])]
    #euleredges[i][len(euleredges[i])-1] 

k = 1
value = {}
while len(value)==0:
    ekmer = kmerFreq(k, path)
    skmer = kmerFreq(k, seq)
    value = { m : skmer[m] for m in set(skmer) - set(ekmer) }
    print value
    k+=1
print k-1

{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{'GTGTTTTTCATCATAA': 1, 'CGGGATGTGGTGAACG': 1, 'TCCATCACCCGCGCCA': 1, 'TATGCCATTCTGAACA': 1, 'AATGCCATTCTGAACC': 1, 'TGACCCGGATGGCGGG': 1, 'GCCATCACCCGCGCCT': 1, 'GGCTCCAGGTCATCCA': 1, 'AAGCCAGCCCGTCAGG': 1, 'AGGGATGTGGTGAACC': 1, 'CATCCGTGACGGGGAG': 1, 'TGCTGCCGGAGAGTGA': 1, 'CAATGCCATTCTGAAC': 1, 'GAATGCCATTCTGAAA': 1, 'CCAGAAAAAAAAGCCG': 1, 'GCAGGACAAATACACG': 1, 'GTGTGAAATATTTCCC': 1, 'ACTCAATTCTCATGAG': 1, 'TATAAAATAAATTTCC': 1, 'GCGGGGTGTACCTGGA': 1, 'CAGCCAGCCCGTCAGC': 1, 'CAAAATCAGCAGCGGA': 1, 'CGCCGACCTGCTGGCA': 1, 'GGGCGCTGGATACGGA': 1, 'GGACCCGGATGGCGGT': 1, 'AAAAATCAGCAGCGGC': 1, 'AATAAAATAAATTTCA': 1, 'TATCCGTGACGGGGAA': 1, 'TGGCGCTGGATACGGC': 1, 'TCAGGACAAATACACC': 1, 'CGCTGCCGGAGAGTGT': 1, 'TTGTGAAATATTTCCG': 1, 'GGATGCGCTGCTGGTT': 1, 'AGATGCGCTGCTGGTC': 1, 'GCAGAAAAAAAAGCCC': 1, 'ACGGGGTGTACCTGGG': 1, 'ATGTTTTTCATCATAT': 1, 'AGCTCCAGGTCATCCG': 1, 'CGGGCTGATACAGCCG': 1, 'AGTTTCGTGAATGTAA': 1, 'GGGGCTGATACAGCCA': 1, 'TGTTTCGTGA

---
**Problem #3:** You will find a BWT of the primary "genome" sequence of *Salmonella Typhimurium* [here](http://csbio.unc.edu/mcmillan/Comp555S18/data/SalmonellaTyphimurium.bwt). This BWT is compressed as follows: Any run of two or more repeated characters is prefixed with an ASCII-encoded number followed by the character. For example the string "AAACGGTTTTTTTTTT" would be encoded as the string "3AC2G10T". What is the compression ratio of this BWT? Where the compression ration is given by: 

$$\frac{len(compressed BWT)}{len(sequence)}$$

What is the average run-length in the BWT (consider characters without runs as run-lengths of 1)?

In [41]:
with open("SalmonellaTyphimurium.bwt.txt") as f:
    content = f.readlines()
content = content[0]
BWTlen = len(content)
seqlen = 0
print content[0:100]
print BWTlen
i = 0
num = ["0","1","2","3","4","5","6","7","8","9"]
letters = 0
while i < BWTlen:
    if content[i] in num:
        if content[i+1] in num:
            temp = int(content[i]+content[i+1])
            seqlen += temp
            letters+=1
            i += 3
        else:
            temp = int(content[i])
            seqlen += temp
            letters+=1
            i += 2
    else:
        seqlen += 1
        letters+=1
        i += 1
print seqlen, letters, float(seqlen)/float(letters), float(BWTlen)/float(seqlen)

A3T3CT2CT2CG3CT4GT2GA3GCG2A3TGATGTGA2GCGT2GT2GC3TCATGCG3C3GA3TCG2CTG3CTGCGCAGATG2CA2TGTCATC2TGTGCGCA
4397999
4857433 3449184 1.40828468415 0.905416297044


**Problem #4:** Uncompress the BWT from in Problem #3, and use code given in class to find how many times the substring "ATGACAACGC" and its reverse complement "GCGTTGTCAT" appear in the *Salmonella Typhimurium* genome. Repeat this for the substring "ATGACAACGCAT" and its reverse complement "ATGCGTTGTCAT". Note these sequences are the first few bases of the *HolC* gene that you searched for in Problem Set #2.

In [36]:
uncompressed = ""
i = 0
while i < len(content):
    if content[i] in num:
        if i+1 < len(content) and content[i+1] in num:
            temp = int(content[i]+content[i+1])
            for j in range(temp):
                uncompressed = uncompressed + content[i+2]
            i += 3
        else:
            temp = int(content[i])
            for j in range(temp):
                uncompressed = uncompressed + content[i+1]
            i += 2
    else:
        uncompressed = uncompressed + content[i]
        i += 1


def FMIndex(bwt):
    fm = [{c: 0 for c in bwt}]
    for c in bwt:
        row = {symbol: count + 1 if (symbol == c) else count for symbol, count in fm[-1].iteritems()}
        fm.append(row)
    offset = {}
    N = 0
    for symbol in sorted(row.keys()):
        offset[symbol] = N
        N += row[symbol]
    return fm, offset

def findBWT(pattern, FMIndex, Offset):
    lo = 0
    hi = len(FMIndex) - 1
    for symbol in reversed(pattern):
        lo = Offset[symbol] + FMIndex[lo][symbol]
        hi = Offset[symbol] + FMIndex[hi][symbol]
    return lo, hi
bwt = uncompressed

FM, Offset = FMIndex(bwt)

x = findBWT("GCGTTGTCAT",FM,Offset)
print x


(3052010, 3052023)


**Problem #5: Programming Problem** 

In class we discussed an algortihm to produce a BWT from a suffix array. In this problem you are asked to write code to do the inverse-- ***produce a suffix array from a BWT***. In the space provided below write your function and test it by finding the genomic indices for all of the substrings than you reported for Problem #4. (Hint: The first implict suffix in the BWT begins with '$', which is the last character of the original string).

In [53]:
# Enter your code here
def resuffix(i, bwt, fmi, offset):
    cur=bwt[i]
    suffix = ''
    
    predecessor =  fmi[i][cur]+ offset[cur] 
    suffix = cur+suffix
    while (predecessor !=i):
        cur=bwt[predecessor]
        
        predecessor = offset[cur] + fmi[predecessor][cur]
        
        suffix = cur+suffix
    return suffix

Click [here to submit](http://csbio.unc.edu/mcmillan/index.py?run=PS.upload) your completed problem set